<a href="https://colab.research.google.com/github/olaviinha/NeuralAudioStemSeparation/blob/master/NeuralAudioStemSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Audio Stem Separation

This is an extension for [Deezer Spleeter](https://github.com/deezer/spleeter) to be run in Google Colab, using Google Drive as data storage.

`input` value may be
1.   Path to an individual audio file located in your Google Drive.
2.   Path to a directory in your Google Drive containing multiple audio files. Each audio file found in the directory will be individually processed.
3.   Single YouTube URL
4.   Multiple YouTube URLs separated by spaces. Each youtube video will be individually processed.

Input will be separated to 4 stems: vocals, drums, bass and other. Stems will be saved as WAV files to your Google Drive. Enter `output_dir` relative to your Google Drive root.

In [ ]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
from google.colab import drive, output
drive.mount('/content/drive')
drive_root = '/content/drive/MyDrive/'

# inhagcutils
!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *
!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json

#@title Install
!apt -q install ffmpeg
!pip -q install spleeter

output.clear()
print('Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json')
op(c.ok, 'Setup finished.')


In [ ]:
#@title Separate

input = "https://www.youtube.com/watch?v=H_qDHs_-Hyw" #@param {type:"string"}
output_dir = "temp" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')
if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir+basename(input))
    separator.separate_to_file(audiofile, seps_dir)
    #!spleeter separate -i "{audiofile}" -o "{output_dir}" -p /content/cfg.json
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir+basename(audiofile)+'/')
elif input_type == "file":
  separator.separate_to_file(input, seps_dir)
  #!spleeter separate -i "{input}" -o "{output_dir}" -p /content/cfg.json
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")